### DATA PREPROCESS - Train Dataset

In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
# print all the outputs in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
pd.options.display.max_colwidth = 5600

In [21]:
def flattenjson(df, var):
    return df[var].apply(json.loads).apply(pd.Series)

In [35]:
def flattenjson2(df, var):
    return df[var].apply(lambda x: "[{'index': '', 'value': ''}]"\
    if x == '[]' else x).apply(lambda x: x[1:-1].replace("'",'"'))\
    .apply(json.loads).apply(pd.Series)

In [22]:
train_chunks = pd.read_csv('train_v2.csv',low_memory=True, chunksize=400000)
totals_chunklist = []
for c in train_chunks:
    totals_chunk = flattenjson(c,'totals')
    totals_chunklist.append(totals_chunk)
    
totals = pd.concat(totals_chunklist)

In [31]:
train_chunks = pd.read_csv('train_v2.csv',low_memory=True, chunksize=400000)
cd_chunklist = []
for c in train_chunks:
    cd_chunk = flattenjson2(c,'customDimensions')
    cd_chunklist.append(cd_chunk)
    
customDimensions = pd.concat(cd_chunklist)

In [36]:
train_chunks = pd.read_csv('train_v2.csv',low_memory=True, chunksize=400000)
device_chunklist = []
for c in train_chunks:
    device_chunk = flattenjson(c,'device')
    device_chunklist.append(device_chunk)
    
device = pd.concat(device_chunklist)

C:\Users\Ty\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [37]:
train_chunks = pd.read_csv('train_v2.csv',low_memory=True, chunksize=400000)
gn_chunklist = []
for c in train_chunks:
    gn_chunk = flattenjson(c,'geoNetwork')
    gn_chunklist.append(gn_chunk)
    
geoNetwork = pd.concat(gn_chunklist)

In [44]:
train_chunks = pd.read_csv('train_v2.csv',low_memory=True, chunksize=400000)
ts_chunklist = []
for c in train_chunks:
    ts_chunk = flattenjson(c,'trafficSource')
    ts_chunklist.append(ts_chunk)
    
trafficSource = pd.concat(ts_chunklist)

C:\Users\Ty\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [43]:
train_nonjson = pd.read_csv('train_v2.csv',
                               usecols=['channelGrouping', 'date','fullVisitorId','socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime'], 
                               low_memory = True)

C:\Users\Ty\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [45]:
train = train_nonjson.join(totals).join(customDimensions)\
.join(device).join(geoNetwork).join(trafficSource)

In [46]:
train.shape

(1708337, 55)

In [47]:
train.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'socialEngagementType',
       'visitId', 'visitNumber', 'visitStartTime', 'bounces', 'hits',
       'newVisits', 'pageviews', 'sessionQualityDim', 'timeOnSite',
       'totalTransactionRevenue', 'transactionRevenue', 'transactions',
       'visits', 'index', 'value', 'browser', 'browserVersion', 'browserSize',
       'operatingSystem', 'operatingSystemVersion', 'isMobile',
       'mobileDeviceBranding', 'mobileDeviceModel', 'mobileInputSelector',
       'mobileDeviceInfo', 'mobileDeviceMarketingName', 'flashVersion',
       'language', 'screenColors', 'screenResolution', 'deviceCategory',
       'continent', 'subContinent', 'country', 'region', 'metro', 'city',
       'cityId', 'networkDomain', 'latitude', 'longitude', 'networkLocation',
       'adContent', 'adwordsClickInfo', 'campaign', 'campaignCode',
       'isTrueDirect', 'keyword', 'medium', 'referralPath', 'source'],
      dtype='object')

In [53]:
train.head()

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,bounces,hits,newVisits,pageviews,...,city,networkDomain,adContent,campaign,campaignCode,isTrueDirect,keyword,medium,referralPath,source
0,Organic Search,20171016,3162355547410993243,1508198450,1,1508198450,1,1,1,1,...,not available in demo dataset,(not set),NaN,(not set),NaN,NaN,water bottle,organic,NaN,google
1,Referral,20171016,8934116514970143966,1508176307,6,1508176307,NaN,2,NaN,2,...,Cupertino,(not set),NaN,(not set),NaN,NaN,NaN,referral,/a/google.com/transportation/mtv-services/bike...,sites.google.com
2,Direct,20171016,7992466427990357681,1508201613,1,1508201613,NaN,2,1,2,...,not available in demo dataset,windjammercable.net,NaN,(not set),NaN,True,NaN,(none),NaN,(direct)
3,Organic Search,20171016,9075655783635761930,1508169851,1,1508169851,NaN,2,1,2,...,not available in demo dataset,unknown.unknown,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
4,Organic Search,20171016,6960673291025684308,1508190552,1,1508190552,NaN,2,1,2,...,not available in demo dataset,prod-infinitum.com.mx,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google


In [48]:
invalid_cols = ['socialEngagementType','browserVersion','browserSize','operatingSystemVersion',\
                'mobileDeviceBranding','mobileDeviceModel',\
        'mobileInputSelector','mobileDeviceInfo','mobileDeviceMarketingName',\
                'screenColors','flashVersion','language',\
         'screenResolution','cityId','latitude','longitude','networkLocation','adwordsClickInfo']

In [51]:
train.drop(columns = invalid_cols,inplace=True)

In [52]:
train.shape

(1708337, 37)

In [54]:
train.to_csv('train_all.csv')